In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

reference:

http://nlp.seas.harvard.edu/annotated-transformer/


### Transformer整体结构

Transformer整体上是encoder-decoder架构。

Encoder把**符号表示序列**映射为**连续表示序列**， $\left(x_{1}, \ldots, x_{n}\right) \to \mathbf{z}=\left(z_{1}, \ldots, z_{n}\right)$

Decoder为每个$\mathbf{z}$中的元素生成输出序列$\left(y_{1}, \ldots, y_{m}\right)$。
在生成下一个符号时，使用先前生成的符号作为额外的输入。

下面代码段展示了encoder-decoder的框架：

In [6]:
class EncoderDecoder(nn.Module):
    '''
    标准的encoder-decoder结构
    '''
    def __ini__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        '''
        src_embed:
        tgt_embed:
        '''
        self.encoder = encoder
        self.decoder = decoder
        
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        
        # decoder中的memory为encode的输出
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
        

In [8]:
class Generator(nn.Module):
    "定义标准的 linear + softmax 生成方式"
    
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)
    
    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)
        

Transformer也采用如上decoder-encoder结构。
在decoder和encoder中分别使用了堆叠的self-attention和point-wise全连接层。



In [ ]:
def clones(Module, N):
    